In [ ]:
!pip freeze > req.txt

In [ ]:
! pip install "camelot-py[base]" -q

In [ ]:
!pip install deepdoctection -q
!pip install timm -q
! apt install tesseract-ocr
! apt install libtesseract-dev
!apt-get install ghostscript -q

In [ ]:
!pip install 'PyPDF2<3.0' -q

In [ ]:
import cv2
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.core.display import HTML
import deepdoctection as dd
import camelot

# Split of every page PDF

In [ ]:
import os
import PyPDF2

def split_pdf(input_pdf_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the PDF file
    with open(input_pdf_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        num_pages = reader.numPages

        # Iterate through each page
        for page_num in range(num_pages):
            page = reader.getPage(page_num)
            writer = PyPDF2.PdfFileWriter()
            writer.addPage(page)

            # Save the page as a separate PDF file
            output_pdf_path = os.path.join(output_folder, f'page_{page_num + 1}.pdf')
            with open(output_pdf_path, 'wb') as output_file:
                writer.write(output_file)

if __name__ == "__main__":
    # Specify the input PDF file path
    input_pdf_path = "/content/tata-steel-report.pdf"

    # Specify the output folder
    output_folder = "/content/savePDF"

    # Call the split_pdf function
    split_pdf(input_pdf_path, output_folder)


# Detection and extraction

In [ ]:
analyzer = dd.get_dd_analyzer()

In [ ]:
file = r"/content/savePDF/page_87.pdf"

In [ ]:
def detectThings(file):
  df = analyzer.analyze(path = file)
  df.reset_state()

  doc = iter(df)
  page = next(doc)

  image = page.viz()
  plt.figure(figsize = (25,17))
  plt.imshow(image)

  return page, doc

page, doc = detectThings(file)

In [ ]:
page_height = page.height
page_width = page.width

In [ ]:
# COLLECTING ALL BOUNDING BOX FROM ALL PAGE

i = 0
bounding_boxes = []

for table in page.tables :
  table = page.tables[i]
  bounding_boxes.append(table.bounding_box)
  i = i + 1

bounding_boxes

[BoundingBox(absolute_coords=True, ulx=178.21522521972656, uly=1581.8509521484375, lrx=2258.066162109375, lry=2751.92529296875, height=1170.0743408203125, width=2079.8509368896484)]

In [ ]:
bounding_box = bounding_boxes[0]

In [ ]:
from camelot import utils
layout, dim = utils.get_page_layout(file)
dim

(594, 738)

In [ ]:
# CONVERSTION

deep_ulx = bounding_box.ulx
deep_uly = bounding_box.uly
deep_lrx = bounding_box.lrx
deep_lry = bounding_box.lry

deep_page_height = page_height
deep_page_width = page_width

camelot_page_width = dim[0]
camelot_page_height = dim[1]

def convert_coordinates(ulx, uly, lrx, lry, deep_width, deep_height, camelot_width, camelot_height):
    # Convert y-coordinates from top-left origin to bottom-left origin
    deep_uly = deep_height - uly
    deep_lry = deep_height - lry

    # Scale coordinates based on page dimensions
    scale_x = camelot_width / deep_width
    scale_y = camelot_height / deep_height

    camelot_ulx = ulx * scale_x
    camelot_uly = deep_uly * scale_y
    camelot_lrx = lrx * scale_x
    camelot_lry = deep_lry * scale_y

    return camelot_ulx, camelot_uly, camelot_lrx, camelot_lry

# Convert coordinates
camelot_ulx, camelot_uly, camelot_lrx, camelot_lry = convert_coordinates(deep_ulx, deep_uly, deep_lrx, deep_lry, deep_page_width, deep_page_height, camelot_page_width, camelot_page_height)

result = [camelot_ulx, camelot_uly, camelot_lrx, camelot_lry]

result

[42.77165405273438, 358.355771484375, 541.93587890625, 77.5379296875]

In [ ]:
pdf_file = file
num_pages = len(pdf_file)

page = 1

left = result[0]
top = result[1]
right = result[2]
bottom = result[3]

table_area = [f'{left},{top},{right},{bottom}']

tables = camelot.read_pdf(pdf_file, pages=f"{page}", flavor='stream', table_areas = table_area)

i = 1

# Process tables from the current page
for table in tables:

    i += 1

    table.to_csv(f'{page}.{i}.csv')

print(tables[0]._bbox)

camelot.plot(tables[0], kind='grid').show()

camelot.plot(tables[0], kind='contour').show()

# Raw Code

In [ ]:
# def convert_coordinates(dd_ulx, dd_uly, dd_lrx, dd_lry, camelot_page_width, camelot_page_height):
#     """Converts Deepdoctection table coordinates to Camelot table coordinates."""

#     # Use exact scaling
#     ulx = round(dd_ulx * camelot_page_width / 4830.0, 2)  # Round to two decimal places
#     uly = round(camelot_page_height - dd_uly * camelot_page_height / 2955.0, 2)
#     lrx = round(dd_lrx * camelot_page_width / 4830.0, 2)
#     lry = round(camelot_page_height - dd_lry * camelot_page_height / 2955.0, 2)

#     # Potential adjustment factor (if needed)
#     # ulx += 0.5  # Example adjustment

#     return [ulx, uly, lrx, lry]

########## using actual deep decteron page size ########################
# def convert_coordinates(dd_ulx, dd_uly, dd_lrx, dd_lry, camelot_page_width, camelot_page_height):

#     # Use exact scaling instead of approximation for accuracy
#     ulx = dd_ulx * camelot_page_width / 4950.0
#     uly = camelot_page_height - dd_uly * camelot_page_height /3075.0
#     lrx = dd_lrx * camelot_page_width / 4950.0
#     lry = camelot_page_height - dd_lry * camelot_page_height / 3075.0

#     return ulx, uly, lrx, lry

# result =  convert_coordinates(189.96823120117188, 463.01727294921875, 2254.64111328125, 2819.728271484375, 1188 ,738)

####################### ONE MORE CODE #############################
# DeepDoctection coordinates
deep_ulx = 189.96823120117188
deep_uly = 463.01727294921875
deep_lrx = 2254.64111328125
deep_lry = 2819.728271484375

# deep_ulx = 2696.837646484375
# deep_uly = 2291.658935546875
# deep_lrx = 4766.8896484375
# deep_lry = 2762.10595703125

# Camelot coordinates
camelot_ulx = 32.5197
camelot_uly = 590
camelot_lrx = 540
camelot_lry = 80

# DeepDoctection page dimensions
deep_page_width = 4830.0
deep_page_height = 2955.0

# Camelot page dimensions
camelot_page_width = 1188.0
camelot_page_height = 738

# Convert DeepDoctection coordinates to Camelot coordinates
def convert_coordinates(ulx, uly, lrx, lry, deep_width, deep_height, camelot_width, camelot_height):
    # Convert y-coordinates from top-left origin to bottom-left origin
    deep_uly = deep_height - uly
    deep_lry = deep_height - lry

    # Scale coordinates based on page dimensions
    scale_x = camelot_width / deep_width
    scale_y = camelot_height / deep_height

    camelot_ulx = ulx * scale_x
    camelot_uly = deep_uly * scale_y
    camelot_lrx = lrx * scale_x
    camelot_lry = deep_lry * scale_y

    return camelot_ulx, camelot_uly, camelot_lrx, camelot_lry

# Convert coordinates
camelot_ulx, camelot_uly, camelot_lrx, camelot_lry = convert_coordinates(deep_ulx, deep_uly, deep_lrx, deep_lry, deep_page_width, deep_page_height, camelot_page_width, camelot_page_height)

# Display Camelot coordinates
print("Converted Camelot Coordinates:")
print("ulx:", camelot_ulx)
print("uly:", camelot_uly)
print("lrx:", camelot_lrx)
print("lry:", camelot_lry)

result = convert_coordinates(deep_ulx, deep_uly, deep_lrx, deep_lry, deep_page_width, deep_page_height, camelot_page_width, camelot_page_height)


In [ ]:
# # You can access the figure's size in inches and dpi (dots per inch).
# figsize_in_inches = fig.get_size_inches()
# dpi = fig.dpi

# # Calculate the figure size in pixels, using the dpi value.
# figsize_in_pixels = figsize_in_inches * dpi

# print("Figure size in inches:", figsize_in_inches)
# print("DPI:", dpi)
# print("Figure size in pixels:", figsize_in_pixels)

# # To get x and y axes limits (the extents of your plot in data coordinates)
# ax = fig.gca()  # Get the current axes instance
# x_limits = ax.get_xlim()  # x-axis limits
# y_limits = ax.get_ylim()  # y-axis limits

# print("x-axis limits:", x_limits)
# print("y-axis limits:", y_limits)

# fig.show()

In [ ]:
# IT IS FOR SEEIING PAGE OF DETECTION IN MATPLOTLIB ACCORDING TO SIZE OF THE PAGE

from matplotlib.ticker import MultipleLocator

page_height = page.height
page_width = page.width

print("Page height:", page_height )
print("Page width:", page_width)

plt.figure(figsize=(25, 17))
plt.imshow(image, extent=(0, page_width, page_height, 0))  # Set the extent to match image dimensions
plt.axis('on')  # If you want to display the axis

# Set the major tick locator for more fine-grained control
ax = plt.gca()  # Get the current axes

# Set major tick locator for x-axis and y-axis
x_major_locator = MultipleLocator(100)  # Change 500 to desired tick interval for x-axis
y_major_locator = MultipleLocator(50)  # Change 500 to desired tick interval for y-axis

ax.xaxis.set_major_locator(x_major_locator)
ax.yaxis.set_major_locator(y_major_locator)

# Optionally turn on the grid for easier analysis
plt.grid(False)

# Draw a red boundary around the image
image_boundary = plt.Rectangle((60, 60), page_width - 120, page_height - 120,
                                linewidth=2, edgecolor='r', fill=False)
ax.add_patch(image_boundary)


plt.show()